# Categorizing urban density

last modified: February, 2016

author: [Raj Singh](https://developer.ibm.com/clouddataservices/author/rrsingh/)

original: https://github.com/ibm-cds-labs/open-data/blob/master/samples/urbanity.ipynb


## Overview

Exploration of an academic study of urban structure and density described in the June 2014 article, ["From Jurisdictional to Functional Analysis of Urban Cores & Suburbs"](http://www.newgeography.com/content/004349-from-jurisdictional-functional-analysis-urban-cores-suburbs) in [new geography](http://www.newgeography.com/). 

## Categories

- Urban (pre-auto urban core): density > 2,900 sq. km
- Auto suburban, early: median house built 1946 to 1979, density < 2,900 sq. km and density > 100 sq. km
- Auto suburban, later: median house built after 1979, density < 2,900 sq. km and density > 100 sq. km
- Auto exurban: all others


In [1]:
import pandas as pd, numpy as np

## Collect U.S. Census data

Census data from the 2013 US Census American Community Survey (ACS), 5-year estimates.

Created from the "zip code tabulation area" (ZCTA) [TIGER/Line® with Selected Demographic and Economic Data product in Geodatabase format](http://www.census.gov/geo/maps-data/data/tiger-data.html). This particular version of the ACS is used for the folowing reasons:

1. 5-year estimates are the most accurate data outside of the decennial census [as explained here](http://www.census.gov/programs-surveys/acs/guidance/estimates.html).
1. 2013 is the most recent data set with 5-year estimates
1. TIGER/Line® gives you the geographic boundaries of the zip codes so you can perform spatial analyses
1. This data set is smaller than the full Census, but still has the important income, education, race, age and occupation demographics we want to use.

If you want to do this yourself, [this article](https://developer.ibm.com/clouddataservices/2015/09/08/census-open-data-on-ibm-cloud/) explains how to get a CSV out of that format.


### Get zip code areas from Census

In [2]:
geo_df = pd.read_csv('file:///Users/rajrsingh/workspace/opendata-service/data/areas.csv.gz', compression='gzip', 
                    usecols=['GEOID10','ALAND10'], dtype={"GEOID10": np.str})
geo_df['GEOID'] = "86000US" + geo_df['GEOID10']
geo_df = geo_df.set_index('GEOID')
geo_df.head()

,GEOID10,ALAND10
GEOID,,
86000US43451,43451,63411475
86000US43452,43452,121783680
86000US43456,43456,9389360
86000US43457,43457,48035540
86000US43458,43458,2573816


### Get population from Census

In [3]:
pop_df = pd.read_csv('file:///Users/rajrsingh/workspace/opendata-service/data/x01_age_sex.csv.gz', compression='gzip', 
                     usecols=['GEOID','B01001e1'], dtype={"GEOID": np.str})
pop_df = pop_df.set_index('GEOID')
pop_df.head()

,B01001e1
GEOID,
86000US01001,17245
86000US01002,29266
86000US01003,11032
86000US01005,5356
86000US01007,14673


### Get housing age from Census

In [4]:
housing_df = pd.read_csv('file:///Users/rajrsingh/workspace/opendata-service/data/x25_housing.csv.gz', compression='gzip', 
                     usecols=['GEOID','B25035e1'], dtype={"GEOID": np.str})
housing_df = housing_df.set_index('GEOID')
housing_df.sample(5)

,B25035e1
GEOID,
86000US94123,1939.0
86000US38401,1981.0
86000US87014,1968.0
86000US97325,1980.0
86000US40385,1983.0


### Join Census data into one DataFrame with nice names

In [5]:
urban_df = geo_df.join(pop_df)
urban_df = urban_df.join(housing_df)

In [6]:
urban_df.columns = ['ZIP','AREAMSQ','Population','MEDYRBUILT']
urban_df.sample(5)

,ZIP,AREAMSQ,Population,MEDYRBUILT
GEOID,,,,
86000US15378,15378,145167,0,NaN
86000US12084,12084,9342451,4429,1974.0
86000US75751,75751,417038419,17235,1978.0
86000US18091,18091,39868967,5932,1971.0
86000US95975,95975,24292428,2005,1983.0


## Density calculation
persons per square km = persons / (area in square meters / 1,000,000)
persons per hectare = persons / (area in square meters / 10,000)

### Compute population density as persons per square kilometer

In [7]:
urban_df['POPPERKMSQ'] = urban_df['Population'] / (urban_df['AREAMSQ']/1000000)
urban_df.sample(4)

,ZIP,AREAMSQ,Population,MEDYRBUILT,POPPERKMSQ
GEOID,,,,,
86000US21231,21231,2256183,16801,1939.0,7446.647723
86000US67544,67544,533670840,3175,1947.0,5.949360
86000US61114,61114,20634652,14358,1981.0,695.819828
86000US76849,76849,2447728509,4005,1971.0,1.636211


### Group population density into 4 categories

In [8]:
urban_df['CAT'] = 'EXURBAN'
urban_df['CAT'][(urban_df['POPPERKMSQ'] >= 2900)] = 'URBAN'
urban_df['CAT'][(urban_df['POPPERKMSQ'] < 2900) & (urban_df['POPPERKMSQ'] >= 100) & (urban_df['MEDYRBUILT'] < 1980) & (urban_df['MEDYRBUILT'] >= 1946)] = 'SUBURBANEARLY'
urban_df['CAT'][(urban_df['POPPERKMSQ'] < 2900) & (urban_df['POPPERKMSQ'] >= 100) & (urban_df['MEDYRBUILT'] >= 1980)] = 'SUBURBANLATE'
urban_df.describe()

/Users/rajrsingh/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/rajrsingh/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/Users/rajrsingh/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/rajrsingh/anaconda/lib/python2.7/site-package

,AREAMSQ,Population,MEDYRBUILT,POPPERKMSQ
count,3.298900e+04,32989.000000,32045.000000,32989.000000
mean,2.250019e+08,9443.177453,1971.068529,487.703913
std,6.575933e+08,13858.010530,15.606758,1912.093435
min,5.094000e+03,0.000000,1939.000000,0.000000
25%,2.351832e+07,719.000000,NaN,7.754449
50%,9.322068e+07,2781.000000,NaN,30.194351
75%,2.304373e+08,12830.000000,NaN,249.247358
max,3.478591e+10,114734.000000,2011.000000,71226.281507


In [9]:
# look at a few records to do a quick sanity check
urban_df.sample(30)

,ZIP,AREAMSQ,Population,MEDYRBUILT,POPPERKMSQ,CAT
GEOID,,,,,,
86000US01151,01151,6134291,9134,1944.0,1489.006635,EXURBAN
86000US12450,12450,25635076,204,1939.0,7.957847,EXURBAN
86000US04265,04265,18292941,996,1971.0,54.447232,EXURBAN
86000US50161,50161,210950140,2058,1958.0,9.755860,EXURBAN
86000US46770,46770,136780386,2883,1956.0,21.077583,EXURBAN
86000US94549,94549,49726775,27784,1962.0,558.733198,SUBURBANEARLY
86000US99147,99147,21199008,131,1995.0,6.179534,EXURBAN
86000US53012,53012,90849846,18868,1972.0,207.683346,SUBURBANEARLY
86000US36455,36455,1392967,502,1939.0,360.381832,EXURBAN


## Mapping Urbanity

In [10]:
#!pip install --user cartodb
# Set up cartodb module for use
from cartodb import CartoDBAPIKey, CartoDBException
# Learn how to use the CartoDB SQL API here:
# http://docs.cartodb.com/cartodb-platform/sql-api/making-calls/
API_KEY = 'dfd11d7ad9ad3f213a6336b5e878865422f624f1'
CARTODB_ACCOUNT = 'ibm'
cl = CartoDBAPIKey(API_KEY, CARTODB_ACCOUNT)

### Update data table in CartoDB

In [11]:
# get all non-exurban zips
notex_df = urban_df[ urban_df['CAT'] != 'EXURBAN' ]
notex_df.head()

,ZIP,AREAMSQ,Population,MEDYRBUILT,POPPERKMSQ,CAT
GEOID,,,,,,
86000US43452,43452,121783680,13659,1974.0,112.157885,SUBURBANEARLY
86000US43458,43458,2573816,312,1960.0,121.220787,SUBURBANEARLY
86000US43460,43460,7158543,6334,1959.0,884.816924,SUBURBANEARLY
86000US43465,43465,28559486,5267,1974.0,184.422087,SUBURBANEARLY
86000US43468,43468,1861424,403,1957.0,216.500915,SUBURBANEARLY


In [ ]:
# generate PostgreSQL INSERT statements for CartoDB
insertsql = ''
for index, row in notex_df.iterrows(): 
    insertsql += "INSERT INTO urbanity(zipcode,category) VALUES ('" + index + "', '" + row['CAT'] + "');"

In [ ]:
# send SQL INSERT statements to CartoDB
try:
    print(cl.sql('delete from urbanity'))
    print(cl.sql(insertsql))
except CartoDBException as e:
    print("some error ocurred", e)

### Urbanity Map!

In [ ]:
%%javascript
element.append("<link rel='stylesheet' href='http://libs.cartocdn.com/cartodb.js/v3/3.15/themes/css/cartodb.css' />")
element.append("<div id='map' style='height:500px;width:900px;padding:0;margin:0'></div>");

require.config({
  paths: {
      cartodblib: 'http://libs.cartocdn.com/cartodb.js/v3/3.15/cartodb'
  }
});

var main = function() {
  cartodb.createVis('map', 'https://ibmanalytics.cartodb.com/u/ibm/api/v2/viz/3a4e9e24-b3f6-11e5-a4c7-0e31c9be1b51/viz.json', {
      shareable: true,title: true,description: true,search: true,tiles_loader: true,
      center_lat: 40, center_lon: -100, zoom: 3
  })
  .done(function(vis, layers) {
    // layer 0 is the base layer, layer 1 is cartodb layer
    // setInteraction is disabled by default
    layers[1].setInteraction(true);
    layers[1].on('featureOver', function(e, latlng, pos, data) {
      cartodb.log.log(e, latlng, pos, data);
    });
    
    // var map = vis.getNativeMap(); // get the native map to work with it
    
  })
  .error(function(err) {
    console.log(err);
  });
}

require(['cartodblib'], main);